# Translate english lyrics to french
We use argosmodel to translate sentences for free offline. You need to download models on the [Google Drive](https://drive.google.com/drive/folders/11wxM3Ze7NCgOk_tdtRjwet10DmtvFu3i) of Argos before.

In [ ]:
import pandas as pd
import os
import pymysql
import re
from os import walk
import unidecode
from pymysql import DataError
from collections import Counter
from langdetect import detect_langs, detect
from argostranslate import package, translate
package.install_from_path('en_fr.argosmodel')
installed_languages = translate.load_installed_languages()
TL = installed_languages[0].get_translation(installed_languages[1])

### Detect lang and insert in database

In [ ]:
connection = pymysql.connect(host='localhost',user='django2',password='password',db='quizz_db',
                             charset='utf8mb4',cursorclass=pymysql.cursors.DictCursor)

with connection.cursor() as cursor:
    cursor.execute(f"SELECT id, artists, name from lyrizz_song")
    songs = cursor.fetchall()
connection.commit()


for raw_song in songs:
    id = raw_song['id']
    artists = raw_song['artists']
    name = raw_song['name']
    with connection.cursor() as cursor:
        cursor.execute(f"SELECT * from lyrizz_lyrics WHERE song_id='{id}'")
        lyrics = cursor.fetchall()
    connection.commit()
    
    count_lang = []
    for raw_lyric in lyrics:
        text = raw_lyric['lyrics_text']
        try:
            tmp_lang = detect(text)
            count_lang.append(tmp_lang)
        except Exception:
            pass
    lang = Counter(count_lang).most_common()[0][0]
    print(f'[{lang}] {artists} - {name}')
    
    
    with connection.cursor() as cursor:
        cursor.execute(f'UPDATE lyrizz_song SET lang="{lang}" WHERE id="{id}"')
        connection.commit()

connection.close()

### Copy lyrics in raw_lyrics_text field

In [ ]:
connection = pymysql.connect(host='localhost',user='django2',password='password',db='quizz_db',
                             charset='utf8mb4',cursorclass=pymysql.cursors.DictCursor)

with connection.cursor() as cursor:
    cursor.execute(f"SELECT id, lyrics_text from lyrizz_lyrics")
    res = cursor.fetchall()
connection.commit()


for raw in res:
    id = raw['id']
    lyrics_text = raw['lyrics_text'].replace('"','\\"')
    
    with connection.cursor() as cursor:
        cursor.execute(f'UPDATE lyrizz_lyrics SET raw_lyrics_text="{lyrics_text}" WHERE id="{id}"')
        connection.commit()

connection.close()

### Translate if necessary and insert in lyrics_text field

In [ ]:
connection = pymysql.connect(host='localhost',user='django2',password='password',db='quizz_db',
                             charset='utf8mb4',cursorclass=pymysql.cursors.DictCursor)

with connection.cursor() as cursor:
    cursor.execute(f"SELECT id, artists, name, lang from lyrizz_song")
    songs = cursor.fetchall()
connection.commit()


for raw_song in songs:
    id = raw_song['id']
    artists = raw_song['artists']
    name = raw_song['name']
    lang = raw_song['lang']
    if lang == 'en':
        with connection.cursor() as cursor:
            cursor.execute(f"SELECT * from lyrizz_lyrics WHERE song_id='{id}'")
            lyrics = cursor.fetchall()
        connection.commit()

        for raw_lyric in lyrics:
            id_lyric = raw_lyric['id']
            text = raw_lyric['raw_lyrics_text']
            new_text = TL.translate(text)
            # Escape non escaped quotes
            new_text = re.sub(r'(?<!\\)"','\\"', new_text)
    
            # /!\ aux guillemets
            try:
                with connection.cursor() as cursor:
                    cursor.execute(f'UPDATE lyrizz_lyrics SET lyrics_text="{new_text}" WHERE id="{id_lyric}"')
                    connection.commit()
            except Exception:
                pass
            
    print(artists, '-', name, id)

connection.close()